In [2]:
import numpy as np
import rustworkx as rx
 
from qiskit.providers import BackendV2, Options
from qiskit.transpiler import Target, InstructionProperties
from qiskit.circuit.library import XGate, YGate, ZGate, RXGate, RYGate, RZGate, CZGate, PhaseGate, TGate, TdgGate, SXGate, SXdgGate, SGate, SdgGate
from qiskit.circuit import Measure, Delay, Parameter, Reset
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_gate_map
import qiskit.qasm2 as qasm2
from functools import partial
import pennylane as qml
from pennylane.tape import QuantumTape
from pennylane_snowflurry.processing import PreProcessor
from pennylane_snowflurry.processing.config import MonarqDefaultConfig
from pennylane_snowflurry.utility.debug import arbitrary_circuit, to_qasm
from pennylane_snowflurry.processing.steps import VF2


from pennylane_snowflurry.monarq_data import connectivity

In [7]:

class FakeMonarQ(BackendV2):
    def __init__(self):
        
        super().__init__(name="Fake MonarQ")
        # Create a heavy-hex graph using the rustworkx library, then instantiate a new target
        self._graph = rx.PyGraph()
        num_qubits = 24
        self._target = Target(
            "Fake MonarQ", num_qubits=num_qubits
        )
 
        # Generate instruction properties for single qubit gates and a measurement, delay,
        #  and reset operation to every qubit in the backend.
        rng = np.random.default_rng(seed=12345678942)
        x_props = {}
        z_props = {}
        measure_props = {}
        delay_props = {}
        
        # Add 1q gates. Globally use virtual rz, x, sx, and measure
        for i in range(num_qubits):
            qarg = (i,)
            z_props[qarg] = InstructionProperties(error=0.0, duration=0.0)
            x_props[qarg] = InstructionProperties(
                error=rng.uniform(1e-6, 1e-4),
                duration=rng.uniform(1e-8, 9e-7),
            )
            measure_props[qarg] = InstructionProperties(
                error=rng.uniform(1e-3, 1e-1),
                duration=rng.uniform(1e-8, 9e-7),
            )
            delay_props[qarg] = None
            
        self._target.add_instruction(SGate(), z_props)
        self._target.add_instruction(SXGate(), x_props)
        self._target.add_instruction(SdgGate(), z_props)
        self._target.add_instruction(SXdgGate(), x_props)
        self._target.add_instruction(XGate(), x_props)
        self._target.add_instruction(YGate(), x_props)
        self._target.add_instruction(ZGate(), z_props)
        self._target.add_instruction(TGate(), z_props)
        self._target.add_instruction(TdgGate(), z_props)
        self._target.add_instruction(PhaseGate(Parameter("theta")), z_props)
        
        self._target.add_instruction(Measure(), measure_props)
        self._target.add_instruction(Reset(), measure_props)
 
        self._target.add_instruction(Delay(Parameter("t")), delay_props)
        # Add chip local 2q gate which is CZ
        cz_props = {}
        for v in connectivity["couplers"].values():
            edge = (v[0], v[1])
            cz_props[edge] = InstructionProperties(
                    error=rng.uniform(7e-4, 5e-3),
                    duration=rng.uniform(1e-8, 9e-7))
                                
        self._target.add_instruction(CZGate(), cz_props)
 
    @property
    def target(self):
        return self._target
 
    @property
    def max_circuits(self):
        return None
 
    @property
    def graph(self):
        return self._graph
 
    @classmethod
    def _default_options(cls):
        return Options(shots=1024)
 
    def run(self, circuit, **kwargs):
        raise NotImplementedError(
            "This backend does not contain a run method"
        )

In [20]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeMonarQ()

ghz = QuantumCircuit(4)
ghz.h(0)
ghz.h(1)
ghz.h(2)
ghz.h(3)
ghz.z(3)
ghz.cx(0, 3)
ghz.cx(2, 3)
ghz.h(0)
ghz.h(1)
ghz.h(2)

op_counts = ghz.count_ops()
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
transpiled_ghz = pm.run(ghz)
ops = [op for op in transpiled_ghz.data]
op_counts = transpiled_ghz.count_ops()

print("Post-Transpilation: ")
print(op_counts)

# print(qasm2.dumps(transpiled_ghz))



Post-Transpilation: 
OrderedDict({'sdg': 10, 'sxdg': 5, 'sx': 2, 'cz': 2})


In [21]:
default_dev = qml.device("default.qubit")

tape = QuantumTape(ops=[qml.Hadamard(0), qml.Hadamard(1), qml.Hadamard(2), qml.Hadamard(3), qml.Z(3), qml.CNOT([0, 3]), qml.CNOT([2, 3]), qml.Hadamard(0), qml.Hadamard(1), qml.Hadamard(2)], measurements=[qml.probs(wires=[0, 1, 2])])

config = MonarqDefaultConfig(False)
tape = PreProcessor.get_processor(config, [0, 1, 2, 3])(tape)[0][0]

node = qml.QNode(lambda : arbitrary_circuit(tape, qml.probs), default_dev)
node()
# print(to_qasm(tape))
specs = qml.specs(node)()
print(specs["resources"].gate_types)



defaultdict(<class 'int'>, {'X90': 5, 'PauliX': 1, 'PauliZ': 2, 'ZM90': 1, 'CZ': 2})
